Import Libraries and hyperparameters

In [678]:
import pandas as pd
import torch.nn as nn
import torch
from torch.nn.functional import normalize

from math import ceil
import time
from tqdm import tqdm

import numpy as np
import statistics

import matplotlib.pyplot as plt
from matplotlib import style

import os
from sys import path as sysPath

# Set your CWD path to folder containing DL Model Architecture.
os.chdir(sysPath[0])
print("CWD: ", os.getcwd())

# NETWORK ARCHITECTURE TO IMPORT [inside myModel folder following the example on top]
import LRM
import importlib

# Reload Deep Learning model if there's any changes to its architecture
importlib.reload(LRM)

print("GPU Avaialble:", torch.cuda.device_count())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

NeuralNetwork = LRM.LogisticRegressionModel().to(device)
print(NeuralNetwork)

CWD:  c:\Coding Related\Hackathon\University Malaya Hackathon
GPU Avaialble: 1
LogisticRegressionModel(
  (fc1): Linear(in_features=6, out_features=300, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=100, out_features=30, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=30, out_features=3, bias=True)
)


Dataset Loading and Preparation

In [679]:
dataset = pd.read_csv("Maternal_Health_Risk_Data_Set_1.csv")

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
x, y  = smote.fit_resample(dataset[["Age", "SystolicBP", "DiastolicBP", "BS", "BodyTemp", "HeartRate", "RiskLevel"]], dataset["RiskLevel"])

# Dataset x is Panda type
dataset = x.to_numpy()
np.random.shuffle(dataset)
np.random.shuffle(dataset)

dataset_row_numbers = dataset.shape[0]
print("Data rows:", dataset_row_numbers)

y_all = []
x_all = []

for row in dataset:
        y_all.append(np.eye(LRM.num_classes)[int(row[6])])
y_all = torch.Tensor(np.array(y_all))

# Normalize data
for row in dataset:
        x = row[0:6].tolist() 
        dev = statistics.stdev(x)
        mean = sum(x) / len(x)   
        x = [(X - mean) / dev for X in x]    

        x_all.append(x)
x_all = torch.Tensor(np.array(x_all))
print(x_all)

VAL_PCT = 0.25
val_size = ceil(len(x_all)*VAL_PCT)

train_X = x_all[:-val_size]
train_y = y_all[:-val_size]

test_X = x_all[-val_size:]
test_y = y_all[-val_size:]

print("train len:", len(train_X))
print("test len:", len(test_X))

Data rows: 1218
tensor([[-1.0584,  1.1092,  0.3351, -1.4101,  0.6226,  0.4015],
        [-0.8606,  1.4367,  0.3428, -1.3419,  0.5178, -0.0948],
        [-0.9950,  0.8353,  0.0278, -1.4042,  1.0507,  0.4854],
        ...,
        [-0.9996,  1.2988,  0.4994, -1.3577,  0.4594,  0.0997],
        [-1.0942,  1.1621,  0.2859, -1.3198,  0.7678,  0.1982],
        [-0.7916,  1.2582,  0.0083, -1.4415,  0.8333,  0.1333]])
train len: 913
test len: 305


Training Process

In [ ]:
BATCH_SIZE = 16
TEST_BATCH_SIZE = 32
EPOCHS = 100
TEST_BATCH_STEP = 10
learning_rate = 0.00005

# optimizer = torch.optim.SGD(NeuralNetwork.parameters(), lr=learning_rate)  
# optimizer = optim.Adam(NeuralNetwork.parameters(), lr=learning_rate)
optimizer = torch.optim.RMSprop(NeuralNetwork.parameters(), lr=learning_rate, alpha=0.9, eps=1e-07)

# Choose loss function
# loss_function = nn.CrossEntropyLoss()
loss_function = nn.MSELoss()


def fwd_pass(X,y, train=False):
    if not train:
        NeuralNetwork.zero_grad()
    
    outputs = NeuralNetwork(X)

    matches = []

    matches = [torch.argmax(i)==torch.argmax(j) for i,j in zip(outputs, y)]
    acc = matches.count(True) / len(matches)   

    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    
    return acc, loss

def test(size):
    random_start = np.random.randint(len(test_X)-size)
    X, y = test_X[random_start:random_start+size], test_y[random_start:random_start+size]
    
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.to(device), y.to(device))
    
    return val_acc, val_loss


def trainModel():    
    counterNow = 0

    with open("currentModelLog.log", "w") as f:
        for epoch in range(EPOCHS):


            print(f"EPOCH {epoch + 1}", end=": ")
            
            for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
                counterNow += 1

                batch_X = train_X[i:i+BATCH_SIZE].to(device)
                batch_y = train_y[i:i+BATCH_SIZE].to(device)
                
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                
                if i % TEST_BATCH_STEP == 0:
                    val_acc, val_loss = test(TEST_BATCH_SIZE)
                    f.write(f"{round(time.time(),3)}, { round(float(acc),2) }, {round(float(loss),2)}, {round(float(val_acc),2)}, {round(float(val_loss),2)}\n")

    return counterNow

counter_runs = trainModel()

In [ ]:
style.use("ggplot")

def createGraph():
    contents = open("currentModelLog.log", "r").read().split("\n")
    
    times = []
    accuracies = []
    losses = []
    
    val_accs = []
    val_losses = []

    for c in contents:
        if c:
            timestamp, acc, loss, val_acc, val_loss = c.split(",")       
            times.append(float(timestamp)) 
            accuracies.append(float(acc)) 
            losses.append(float(loss)) 
            val_accs.append(float(val_acc)) 
            val_losses.append(float(val_loss)) 
    
    ax1 = plt.subplot2grid((2,1), (0,0))
    ax2 = plt.subplot2grid((2,1), (1,0))
    ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)
    
    ax1.plot(times, accuracies, label="T Accuracy")
    ax1.plot(times, val_accs, label="V Accuracy")
    ax1.legend(loc=2)
    
    ax2.plot(times, losses, label="T Loss")
    ax2.plot(times, val_losses, label="V Loss")
    ax2.legend(loc=2)
    
    plt.show()
    print("Last Test Loss:", val_losses[-1])
    print("Last Test Acc:", val_accs[-1])
    
createGraph()

In [ ]:
import random

TEST_NUM = random.randint(0, len(test_X))

print(test_X[TEST_NUM].shape)
print(torch.tensor([test_X[TEST_NUM].tolist()]).shape)

prediction = NeuralNetwork(torch.tensor([test_X[TEST_NUM].tolist()]).to(device))

print(prediction)

answer = torch.argmax(prediction[0]).item()
prob = prediction[0][answer].item()

print("ANSWER INDEX:", answer)
print("ANSWER PROBABILITY:", prob)
print("CORRECT ANSWER:", test_y[TEST_NUM])

if(test_y[TEST_NUM][answer] == 1):
    print(True)
else:
    print(False)

In [ ]:
torch.save(NeuralNetwork.state_dict(), "model.pt")

pytorch_model = LRM.LogisticRegressionModel()
pytorch_model.load_state_dict(torch.load("model.pt"))
pytorch_model.eval()
dummy_input = torch.tensor([torch.zeros(6).tolist()])
torch.onnx.export(pytorch_model, dummy_input, "onnx_model.onnx", verbose=True)

print(dummy_input.shape)